<a href="https://colab.research.google.com/github/pythonkvs/seminars/blob/main/%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80_%D0%B8%D1%82%D0%B5%D1%80%D0%B0%D1%82%D0%BE%D1%80%D1%8B_%D0%B3%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%82%D0%BE%D1%80%D1%8B_30_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://compscicenter.ru/courses/python/2015-autumn/classes/1542/

# Итераторы

В отличие от, например, Java в Python нет чёткого
разделения на iterable и iterator: протокол итераторов
покрывает оба случая.

## Протокол итераторов

Протокол итераторов состоит из двух методов:
*   Метод `__iter__` возвращает экземпляр класса,
реализующего протокол итераторов, например, `self`.
*   Метод `__next__` возвращает следующий по порядку
элемент итератора. Если такого элемента нет, то метод
должен поднять исключение `StopIteration`.

## Коллекции и итераторы

* Для коллекций обычно нет смысла реализовывать
протокол итераторов целиком, достаточно реализовать
только метод `__iter__`.

* Иногда элементы коллекции можно перечислить более
чем одним способом. В этом случае удобно реализовывать
дополнительные методы, возвращающие итераторы:
```python
class BinaryTree:
    def __iter__(self):
        return self.inorder_iter()
    def preorder_iter(self):
        # ...
    def inorder_iter(self):
        return InOrderIterator(self)
    def postorder_iter(self):
        # ...
```

## Функции `iter` и `next`

У функции `iter` две формы вызова:

*   принимает объект и вызывает у него метод `__iter__`,
*   принимает функцию и терминальное значение и вызывает
функцию до тех пор, пока она не вернёт нужное значение:
```python
from functools import partial
with open(path, "rb") as handle:
        read_block = partial(handle.read, 64)
        for block in iter(read_block, ""):
            do_something(block)
```

Функция `next` принимает итератор и вызывает у него
метод `__next__`. Можно также указать значение, которое
нужно вернуть в случае возникновения исключения
`StopIteration`:

In [1]:
next(iter([1, 2, 3]))

1

In [2]:
next(iter([]), 42)

42

## Итераторы часто используются неявно, цикл `for`

Как выглядит для нас цикл `for`:

In [3]:
for i in 'seq':
    print(i)

s
e
q


Как он работает на самом деле (процесс исполнения оператора `for` можно концептуально
записать так):

In [4]:
iterator = iter('seq')
while True:
    try:
        i = next(iterator)
    except StopIteration:
        break
    print(i)

s
e
q


## Операторы `in` и `not in`

*   Операторы `in` и `not in` используют “магический” метод
`__contains__`, который возвращает `True`, если переданный
элемент содержится в экземпляре класса.
*   По умолчанию метод `__contains__` реализован через
протокол итераторов:
```python
class object:
        # ...

        def __contains__(self, target):
            for item in self:
                if item == target:
                    return True
            return False
```
*   Пример:

In [5]:
lst = [5, 39]
5 in lst # ≡ lst.__contains__(5)

True

In [6]:
42 not in lst # ≡ not lst.__contains__(42)

True

## Упрощенный вариант реализации протокола итераторов с использованием метода `__getitem__`

*   Метод `__getitem__` принимает один аргумент — индекс
элемента в последовательности и
    *   либо возвращает элемент, соответствующий индексу,
    *   либо поднимает `IndexError`, если элемента с таким
индексом нет.
*   Пример:

In [7]:
class Identity:
    def __getitem__(self, idx):
        if idx > 5:
            raise IndexError(idx)
        return idx
list(Identity())

[0, 1, 2, 3, 4, 5]

## “Семантика” упрощенного протокола итераторов: `seq_iter`

In [8]:
class seq_iter:
    def __init__(self, instance):
        self.instance = instance
        self.idx = 0

    def __iter__(self):
        return self
        
    def __next__(self):
        try:
            res = self.instance[self.idx]
        except IndexError:
            raise StopIteration
        
        self.idx += 1
        return res

## “Семантика” упрощенного протокола итераторов: `object`

In [9]:
class object:
    # ...
    def __iter__(self):
        if not hasattr(self, "__getitem__"):
            cls = self.__class__
            msg = "{} object is not iterable"
            raise TypeError(msg.format(cls.__name__))
        return seq_iter(self)

## Резюме: итераторы

*   В Python нет чёткого различия между *iterable* и *iterator*.
*   Итератор — это экземпляр класса, который реализует два
метода `__iter__` и `__next__`.
*   Альтернативно можно воспользоваться реализацией этих
методов по умолчанию и определить метод `__getitem__`.
*   Протокол итераторов используется:
    *   оператором `for`,
    *   операторами `in` и `not in`.
*   Протокол итераторов реализуется всеми встроенными
коллекциями, а также, например, файлами и объектами
типа `map`, `filter` и `zip`.

# Генераторы

* Генератор — это функция, которая использует не только
оператор `return`, но и оператор `yield`.
* В результате выполнения оператора `yield` работа функции
приостанавливается, а не прерывается, как при
использовании оператора `return`.

In [10]:
def g():
    print("Started")
    x = 42
    yield x
    x += 1
    yield x
    print("Done")

In [11]:
type(g)

function

In [12]:
gen = g()
type(gen)

generator

In [13]:
next(gen)

Started


42

In [14]:
next(gen)

43

* В генераторе есть внутреннее изменяемое состояние в виде локальных переменных, которое он хранит автоматически.
* Генератор — более простой способ создания собственного итератора, чем его прямое определение.
* Все генераторы являются итераторами, но не наоборот.

- Примеры функций-генераторов:
    - `zip`
    - `enumerate`
    - `reversed`
    - `map`
    - `filter`


## Ключевое слово `yield`

- `yield` — это слово, по смыслу похожее на `return`.<br><br>
- Но используется в функциях, возвращающих генераторы.<br><br>
- При вызове такой функции тело не выполняется, функция только возвращает генератор.<br><br>
- В первый запуск функция будет выполняться от начала и до `yield`.<br><br>
- После выхода состояние функции сохраняется.<br><br>
- На следующий вызов будет проводиться итерация цикла и возвращаться следующее значение.<br><br>
- И так далее, пока не кончится цикл каждого `yield` в теле функции.<br><br>
- После этого генератор станет пустым.

## Примеры генераторов

In [15]:
def my_range(n):
    yield 'You really want to run this generator?'

    i = -1
    while i < n:
        i += 1
        yield i

In [16]:
gen = my_range(3)
while True:
    try:
        print(next(gen), end='   ')
    except StopIteration:  # we want to catch this type of exceptions
        break

You really want to run this generator?   0   1   2   3   

In [17]:
for e in my_range(3):
    print(e, end='   ')

You really want to run this generator?   0   1   2   3   

### Примеры генераторов: `unique`

In [18]:
def unique(iterable, seen=None):
    seen = set(seen or [])
    for item in iterable:
        if item not in seen:
            seen.add(item)
            yield item

In [19]:
xs = [1, 1, 2, 3]
unique(xs)

<generator object unique at 0x7f8e26b185d0>

In [20]:
list(unique(xs))

[1, 2, 3]

In [21]:
1 in unique(xs)

True

### Примеры генераторов: `chain`

In [22]:
def chain(*iterables):
    for iterable in iterables:
        for item in iterable:
            yield item

In [23]:
xs, ys = range(3), [42]
chain(xs, ys)

<generator object chain at 0x7f8e26b189d0>

In [24]:
list(chain(xs, ys))

[0, 1, 2, 42]

In [25]:
42 in chain(xs, ys)

True

## Переиспользование генераторов

Основное правило переиспользования генераторов:  **не
делайте этого**.

In [26]:
def g():
    yield 42

In [27]:
gen = g()
list(gen)

[42]

In [28]:
list(gen) # не тут-то было!

[]

* Если вы хотите переиспользовать генератор, подумайте
ещё раз.
* Если вы уверены, что без переиспользования не обойтись,
воспользуйтесь функцией `tee` из модуля `itertools`.

## Коллекции и генераторы

* Генераторы позволяют компактно реализовывать метод
`__iter__` у коллекций.
* Рассмотрим уже знакомый нам класс бинарного дерева:
```python
class BinaryTree:
    def __init__(self, value, left=(), right=()):
        self.value = value
        self.left, self.right = left, right
        
    def __iter__(self): # inorder
        for node in self.left:
            yield node
        yield self.value
        for node in self.right:
            yield node
```
* Плюс генераторов в том, что они позволяют обойтись без
лишних классов, например, `InOrderIterator`.

## Выражения-генераторы

* Напоминание: в Python есть генераторы списков, множеств
и словарей.
* Выражения-генераторы работают аналогичным образом,
но не порождают коллекцию в процессе работы:

In [29]:
gen = (x ** 2 for x in range(10) if x % 2 == 1)
gen

<generator object <genexpr> at 0x7f8e26b188d0>

In [30]:
next(gen)

1

In [31]:
list(gen)

[9, 25, 49, 81]

Если выражение-генератор — единственный аргумент
функции, скобки можно опустить:

In [32]:
list(filter(lambda x: x % 2 == 1,
            (x ** 2 for x in range(10))))

[1, 9, 25, 49, 81]

In [33]:
sum(x ** 2 for x in range(10) if x % 2 == 1)

165

## Выражение `yield`

Оператор `yield` можно использовать как выражение:

In [34]:
def g():
    res = yield # точка входа 1
    print("Got {!r}".format(res))
    res = yield 42 # точка входа 2
    print("Got {!r}".format(res))

In [35]:
gen = g()
next(gen) # "промотаем" до первого yield

In [36]:
next(gen) # "промотаем" до второго yield

Got None


42

In [37]:
next(gen) # выполним оставшуюся часть генератора

Got None


StopIteration: ignored

На первый взгляд выражение `yield` выглядит бесполезно,
но первое впечатление обманчиво.

## Интерфейс генераторов: `send`

Метод `send` возобновляет выполнение генератора и
“отправляет” свой аргумент в следующий `yield`:

In [38]:
gen = g()
gen.send("foobar")

TypeError: ignored

Чтобы инициализировать генератор нужно “отправить” ему
`None`. Функция `next` делает ровно это:

In [39]:
gen = g()
next(gen)

Результатом метода `send` является следующее значение
генератора или исключение `StopIteration`, если такого
исключения нет.

In [40]:
gen = g()
gen.send(None) # ≡ next(gen)
gen.send("foobar")

Got 'foobar'


42

## Интерфейс генераторов: `throw`

Метод `throw` поднимает переданное исключение в месте,
где генератор приостановил исполнение и возвращает
следующее значение генератора.

In [41]:
def g():
    try:
        yield 42
    except Exception as e:
        yield e

In [42]:
gen = g()
next(gen)

42

In [43]:
gen.throw(ValueError, "something is wrong")

ValueError('something is wrong')

In [44]:
gen.throw(RuntimeError, "another error")

RuntimeError: ignored

Если генератор не обработал брошенное в него
исключение, то выполнение генератора прекращается и
исключение передаётся наверх по стеку вызовов.

## Интерфейс генераторов: `close`

Метод `close` поднимает специальное исключение
`GeneratorExit` в месте, где генератор приостановил
исполнение:

In [45]:
def g():
    try:
        yield 42
    finally:
        print("Done")

In [46]:
gen = g()
next(gen)

42

In [47]:
gen.close()

Done


* Если всё хорошо, то метод `close` завершает работу
генератора и ничего не возвращает.
* Что может пойти не так? Генератор может обработать
исключение `GeneratorExit` и вернуть значение с помощью
`yield` или поднять другое исключение.

## Генераторы как сопрограммы *aka* coroutines

*   Сопрограмма — это программа, которая может иметь
больше одной точки входа, а также поддерживает
остановку и продолжение с сохранением состояния.
*   Звучит как определение генератора наоборот:

In [48]:
def grep(pattern):
    print("Looking for {!r}".format(pattern))
    while True:
        line = yield
        if pattern in line:
            print(line)

In [49]:
gen = grep("Gotcha!")
next(gen)

Looking for 'Gotcha!'


In [50]:
gen.send("This line doesn't have \
        what we're looking for")

In [51]:
gen.send("This one does. Gotcha!")

This one does. Gotcha!


* Прежде, чем начать работать с сопрограммой, её нужно
инициализировать с помощью вызова функции `next`.
* Объявим декоратор `coroutine`, который скроет эту деталь
реализации:

In [52]:
import functools
def coroutine(g):
    @functools.wraps(g)
    def inner(*args, **kwargs):
        gen = g(*args, **kwargs)
        next(gen)
        return gen
    return inner

In [53]:
grep = coroutine(grep)
gen = grep("Gotcha!")
gen.send("One more line for ya!")

Looking for 'Gotcha!'


## Оператор `yield from`

Оператор `yield from` позволяет делегировать выполнение
другому генератору:

In [54]:
def chain(*iterables):
    for iterable in iterables:
        yield from iterable
        
        # for item in iterable:
        #     yield item

Любые вызовы методов `send` и `throw` у родительского
генератора будут переданы вложенному генератору без
изменений.

## Оператор `return` и исключение `StopIteration`

* Кроме оператора `yield` в теле генератора можно
использовать оператор `return`.
* На человеческом языке использование return означает: «У
меня больше нет элементов, извини, возьми лучше вот
это.»
* На Python “вот это”, превращается в аргумент уже
знакомого нам исключения `StopIteration`:

In [55]:
def g():
    yield 42
    return []

In [56]:
gen = g()
next(gen)

42

In [57]:
next(gen)

StopIteration: ignored

## `return` ≠ `raise StopIteration`

Несмотря на схожесть, использование оператора `return` в
генераторе не эквивалентно поднятию исключения
`StopIteration`.  
Контрпример:

In [58]:
def g():
    try:
        yield 42
        raise StopIteration([]) # ≠ return []
    except Exception as e:
        pass

## Выражение `yield from`

* Оператор `yield from`, как и оператор `yield`, можно
использовать в качестве выражения.
* При этом значением выражения `yield from` будет
значение атрибута `value` у поднятого вложенным
генератором исключения `StopIteration`:

In [59]:
 def f():
    yield 42
    return []

def g():
    res = yield from f()
    print("Got {!r}".format(res))

In [60]:
gen = g()
next(gen)

42

In [61]:
# next(gen)
next(gen, None)

Got []


## Менеджеры контекста и генераторы: мотивация

* Протокол менеджеров контекста требует реализации двух
методов: `__enter__` и `__exit__`,
* Если мы хотим, чтобы у менеджера было какое-то
состояние, то мы вынуждены также добавить метод
`__init__`.
* В итоге получаем:

In [62]:
class cd:
    def __init__(self, path):
        self.path = path

    def __enter__(self):
        self.saved_cwd = os.getcwd()
        os.chdir(self.path)

    def __exit__(self, *exc_info):
        os.chdir(self.saved_cwd)

## Менеджеры контекста и генераторы: `@contextmanager`

Декоратор `contextmanager` из модуля `contextlib`
принимает генератор специального вида и строит по нему
менеджер контекста.

In [63]:
from contextlib import contextmanager
@contextmanager
def cd(path):              # __init__
    old_path = os.getcwd() # __enter__
    os.chdir(path)
    try:
        yield              # ---------
    finally:
        os.chdir(old_path) # __exit__

Генераторы позволяют сократить количество
синтаксического шума при реализации менеджеров
контекста.

## Ещё один пример использования `@contextmanager`

Метод `__enter__`, построенный декоратором `contextmanager`,
возвращает аргумент оператора `yield`:

In [64]:
from contextlib import contextmanager
import tempfile
import shutil
@contextmanager
def tempdir():                  # __init__
    outdir = tempfile.mkdtemp() # __enter__
    try:
        yield outdir            # ---------
    finally:
        shutil.rmtree(outdir)   # __exit__

with tempdir() as path:
    print(path)

/tmp/tmp1ryw8ey7


## Генераторы: резюме

*   Генератор в Python — это функция, которая использует
операторы `yield` или `yield from`.
*   В мире Python генераторы вездесущи не менее, чем
любимые всеми декораторы.
*   Мы поговорили о том, что генераторы можно использовать
    *   как итераторы,
    *   как сопрограммы,
    *   для компактной реализации менеджеров контекста.

# Модуль `itertools`


- Модуль представляет собой набор инструментов для работы с итераторами и последовательностями.<br><br>
- Содержит три основных типа итераторов:<br><br>
    - бесконечные итераторы,
    - конечные итераторы,
    - комбинаторные итераторы.<br><br>

- Позволяет эффективно решать небольшие задачи вида:<br><br>
    - итерирование по бесконечному потоку,
    - слияние в один список вложенных списков,
    - генерация комбинаторного перебора сочетаний элементов последовательности,
    - аккумуляция и агрегация данных внутри последовательности.

## Модуль `itetools`: примеры

### `count`

In [65]:
from itertools import count

for i in count(start=0):
    print(i, end='  ')
    if i == 5:
        break

0  1  2  3  4  5  

### `cycle`

In [66]:
from itertools import cycle
 
n = 0
for item in cycle('XYZ'):
    if n > 4:
        break
    print(item, end='  ')
    n += 1

X  Y  Z  X  Y  

### `accumulate`

In [67]:
from itertools import accumulate

for i in accumulate(range(1, 5), lambda x, y: x * y):
    print(i)

1
2
6
24


### `chain`

In [68]:
from itertools import chain

for i in chain([1, 2], [3], [4]):
    print(i)

1
2
3
4


In [69]:
from itertools import chain

chain_two = chain(range(2), range(5, 10))
print(chain_two)
list(chain_two)

[0, 1, 5, 6, 7, 8, 9]

### `groupby`

In [70]:
from itertools import groupby
 
vehicles = [('Ford', 'Taurus'), ('Dodge', 'Durango'),
            ('Chevrolet', 'Cobalt'), ('Ford', 'F150'),
            ('Dodge', 'Charger'), ('Ford', 'GT')]
 
sorted_vehicles = sorted(vehicles)

print(sorted_vehicles)
print()
 
for key, group in groupby(sorted_vehicles, lambda x: x[0]):
    for maker, model in group:
        print('{model} is made by {maker}'.format(model=model, maker=maker))
    
    print ("**** END OF THE GROUP ***\n")

[('Chevrolet', 'Cobalt'), ('Dodge', 'Charger'), ('Dodge', 'Durango'), ('Ford', 'F150'), ('Ford', 'GT'), ('Ford', 'Taurus')]

Cobalt is made by Chevrolet
**** END OF THE GROUP ***

Charger is made by Dodge
Durango is made by Dodge
**** END OF THE GROUP ***

F150 is made by Ford
GT is made by Ford
Taurus is made by Ford
**** END OF THE GROUP ***



## Модуль `itertools`: `islice`

Функция `islice` обобщает понятие слайса на
произвольный итератор:

In [71]:
from itertools import islice
xs = range(10)
list(islice(xs, 3)) # ≡ xs[:3]

[0, 1, 2]

In [72]:
list(islice(xs, 3, None)) # ≡ xs[3:]

[3, 4, 5, 6, 7, 8, 9]

In [73]:
list(islice(xs, 3, 8, 2)) # ≡ xs[3:8:2]

[3, 5, 7]

## Модуль `itertools`: бесконечные итераторы

Для удобства реализуем функцию `take`, которая строит список из более чем `n`
первых элементов переданного ей итератора.

In [74]:
from itertools import islice
def take(n, iterable):
    return list(islice(iterable, n))

In [75]:
take(3, range(10))

[0, 1, 2]

Названия бесконечных итераторов говорят сами за себя:

In [76]:
from itertools import count, cycle, repeat
take(3, count(0, 5))

[0, 5, 10]

In [77]:
take(5, cycle([1, 2, 3]))

[1, 2, 3, 1, 2]

In [78]:
take(3, repeat(42))

[42, 42, 42]

In [79]:
take(6, repeat(42, 2))

[42, 42]

## Модуль `itertools`: `dropwhile` и `takewhile`

* Функции `dropwhile` и `takewhile` обобщают логику функций
`drop` и `take` на произвольный предикат.
* Обратите внимание, что обе функции возвращают
итератор, а не список, как реализованная нами функция
`take`:

In [80]:
from itertools import dropwhile, takewhile
list(dropwhile(lambda x: x < 5, range(10)))

[5, 6, 7, 8, 9]

In [81]:
it = takewhile(lambda x: x < 5, range(10))
it

In [82]:
list(it)

[0, 1, 2, 3, 4]

## Модуль `itertools`: `chain`

В модуле `itertools` реализован 
генератор `chain`, который конкатенирует произвольное
число итераторов:

In [83]:
from itertools import chain
take(5, chain(range(2), range(5, 10)))

[0, 1, 5, 6, 7]

Сконкатенировать итератор итераторов (!) можно с
помощью метода `chain.from_iterable`:

In [84]:
it = (range(x, x ** x) for x in range(2, 4))
take(5, chain.from_iterable(it))

[2, 3, 3, 4, 5]

## Модуль `itertools`: `tee`

Функция `tee` создаёт `n` независимых копий переданного ей
итератора:

In [85]:
from itertools import tee
it = range(3)
a, b, c = tee(it, 3)
list(a), list(b), list(c)

([0, 1, 2], [0, 1, 2], [0, 1, 2])

Использовать `it` после копирования не рекомендуется,
потому что в этом случае скопированные итераторы
`a`, `b`, `c` могут пропустить элемент:

In [86]:
it = iter(range(3))
a, b = tee(it, 2)
used = list(it)
list(a), list(b)

([], [])

## Модуль `itertools`: комбинаторные итераторы

В модуле `itertools` в виде итераторов реализованы полезные
комбинаторные операции, например:  
* декартово произведение итераторов,

In [87]:
import itertools
list(itertools.product("AB", repeat=2))

[('A', 'A'), ('A', 'B'), ('B', 'A'), ('B', 'B')]

In [88]:
list(itertools.product("AB", repeat=3))

[('A', 'A', 'A'),
 ('A', 'A', 'B'),
 ('A', 'B', 'A'),
 ('A', 'B', 'B'),
 ('B', 'A', 'A'),
 ('B', 'A', 'B'),
 ('B', 'B', 'A'),
 ('B', 'B', 'B')]

* перестановки элементов итератора,

In [89]:
list(itertools.permutations("AB"))

[('A', 'B'), ('B', 'A')]

* сочетания (с повторениями и без) из элементов итератора.

In [90]:
from itertools import combinations, \
    combinations_with_replacement
list(combinations("ABC", 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [91]:
list(combinations_with_replacement("ABC", 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

## Модуль `itertools`: резюме

* Модуль `itertools` предоставляет обширный набор
компонент для реализации операций над
последовательностями.
* Мы обсудили:
    * `islice`,
    * бесконечные итераторы `count`, `cycle`, `repeat`,
    * `chain`,
    * `tee`,
    * `groupby`,
    * `accumulate`,

    * комбинаторные итераторы `product`, `permutations`,
`combinations` и `combinations_with_replacement`.